In [1]:
from glob import glob
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
import sys
import time
import datetime
from tqdm import tqdm
import yaml
import random
import os
import gc
import cv2

from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW, lr_scheduler
from torch.distributions import Uniform
from torch.utils.data import DataLoader, Dataset

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet34, resnet50

import sys
sys.path.append('..')
from libs import transform as tr
from libs import spectrogram as spec
from libs import criterion as cr
from libs import wormup as wu


import warnings
warnings.filterwarnings("ignore")

/home/yuigahama/anaconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore

set_seed(42)

In [3]:
with open('../settings/efficient_b3_base.yaml') as f:
    settings = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
def normalize_melspec(X: np.ndarray):
    eps = 1e-6
    mean = X.mean()
    X = X - mean
    std = X.std()
    Xstd = X / (std + eps)
    norm_min, norm_max = Xstd.min(), Xstd.max()
    if (norm_max - norm_min) > eps:
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def save(fold, model, optim, criterion, file_path="../../model/"):
    if not settings['test_name'] in os.listdir(file_path):
        os.mkdir(file_path+settings['test_name'])
    
    
    output_path = file_path + settings['test_name'] + '/' + f"{settings['test_name']}_{fold}.model"
    
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.cpu().state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'criterion': criterion
        },
        output_path)
    
    model.to(device)
    
    return output_path

In [5]:
class RfcxDataSet(Dataset):
    def __init__(self,
                 tp:pd.DataFrame,
                 train: bool,
                 data_path:str,
                 transform_params:dict,
                 pre_calc=True,
                 n_mels=128
    ):
        self.tp = tp
        self.path = data_path
        self.img_size = 256
        self.train = train
        self.n_mels = n_mels
        self.pre_calc = pre_calc
        
        self.transform = tr.Compose([
            #tr.OneOf([
            #    tr.GaussianNoiseSNR(min_snr=transform_params['GaussianNoiseSNR']),
            #    tr.PinkNoiseSNR(min_snr=transform_params['PinkNoiseSNR'])
            #]),
            tr.PitchShift(max_steps=transform_params['max_steps'], sr=SR),
            #tr.TimeStretch(),
            #tr.TimeShift(sr=sr),
            tr.VolumeControl(mode="sine")
        ])
        
        
    def __len__(self):
        return len(self.tp)
    
    def load(self, record_path):
        y, orig_sr = sf.read(record_path)
        
        if orig_sr != SR:
            y = librosa.resample(y, orig_sr=orig_sr, target_sr=SR, res_type="kaiser_best")
        return y
    
    def get_random_duration(self, duration=10):
        start_sec = random.randint(0, 60-duration)
        end_sec = start_sec + 10
            
        return start_sec, end_sec
    
    def get_duration(self, t_min, t_max, duration=10):
        annotated_duration = t_max - t_min
        
        if annotated_duration > duration:
            limit_sec = t_max - duration
            start_sec = random.randint(t_min, limit_sec)
            end_sec = start_sec + duration

        else:
            res_time = duration - annotated_duration
            front_limit = res_time if res_time < t_min else t_min
            
            front_time = random.randint(0, front_limit)
            
            back_limit = 60 - t_max
            
            tmp_time = res_time - front_time
            back_time = tmp_time if tmp_time < back_limit else back_limit
            
            if not tmp_time < back_limit:
                front_time += tmp_time - back_limit
            
            start_sec = t_min - front_time
            end_sec = t_max + back_time
            
        return start_sec, end_sec
    
    def create_mel(self, y):
        y = self.transform(y)
        
        melspec = librosa.feature.melspectrogram(
            y,
            sr=SR,
            fmin=0,
            fmax=15000,
            n_mels=128
        )

        pcen = librosa.pcen(melspec, sr=SR, **self.pcen_parameters)
        clean_mel = librosa.power_to_db(melspec ** 1.5)
        melspec = librosa.power_to_db(melspec)

        norm_melspec = normalize_melspec(melspec)
        norm_pcen = normalize_melspec(pcen)
        norm_clean_mel = normalize_melspec(clean_mel)

        image = np.stack([norm_melspec, norm_pcen, norm_clean_mel], axis=-1)

        return image
    
    def __getitem__(self, idx: int):
        sample = self.tp.iloc[idx, :]
        recording_id = sample['recording_id']
        t_min = int(round(sample['t_min']))
        t_max = int(round(sample['t_max']))
        
        start_sec, end_sec = self.get_duration(t_min, t_max, settings['duration'])
            
        record_path = self.path + recording_id + '.flac'
        y = self.load(record_path)
        y =  y[start_sec*SR:end_sec*SR]
        
        if self.train:
            y = self.transform(y)
        
        species_id = sample['species_id']
        target = torch.zeros([24], dtype=torch.float32)
        target[species_id] = 1
        
        return y, target

In [6]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


def init_weights(model):
    classname = model.__class__.__name__
    if classname.find("Conv2d") != -1:
        nn.init.xavier_uniform_(model.weight, gain=np.sqrt(2))
        model.bias.data.fill_(0)
    elif classname.find("BatchNorm") != -1:
        model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)
    elif classname.find("GRU") != -1:
        for weight in model.parameters():
            if len(weight.size()) > 1:
                nn.init.orghogonal_(weight.data)
    elif classname.find("Linear") != -1:
        model.weight.data.normal_(0, 0.01)
        model.bias.data.zero_()


def do_mixup(x: torch.Tensor, mixup_lambda: torch.Tensor):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = (x[0::2].transpose(0, -1) * mixup_lambda[0::2] +
           x[1::2].transpose(0, -1) * mixup_lambda[1::2]).transpose(0, -1)
    return out


class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)

        return torch.from_numpy(np.array(mixup_lambdas, dtype=np.float32))


def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output

In [7]:
class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.clamp(self.att(x), -10, 10), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)


class AttBlockV2(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

In [8]:
class EfficientNetSED(nn.Module):
    def __init__(
        self,
        base_model_name: str,
        pretrained=False,
        num_classes=24,
        spectrogram_params={},
        logmel_extractor_params={},
        spec_augmenter_params={},
        pce_params={}
    ):
        super().__init__()
        self.spectrogram_extractor = Spectrogram(**spectrogram_params)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(**logmel_extractor_params)
        
        #Pcen converter
        self.pcen_converter = spec.pcen(**pce_params)

        # Spec augmenter
        #self.spec_augmenter = SpecAugmentation(**spec_augmenter_params)
        
        self.interpolate_ratio = 30  # Downsampled ratio
        self.mixup_alpha = 0.2
        self.random_state = np.random.RandomState(123)
        
        if pretrained:
            self.base_model = EfficientNet.from_pretrained(base_model_name)
        else:
            self.base_model = EfficientNet.from_name(base_model_name)

        in_features = self.base_model._fc.in_features

        self.fc1 = nn.Linear(in_features, in_features, bias=True)
        self.att_block = AttBlockV2(in_features, num_classes, activation="sigmoid")

        self.init_weight()
        
    def mixup(self, x):
        sizws = x.size()
        #lam = torch.from_numpy(self.random_state.beta(self.mixup_alpha, self.mixup_alpha, (sizws[0], 1))).cuda()
        lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
        index = list(range(x.size(0)))
        random.shuffle(index)
        #out = (x.view(sizws[0], -1) * lam + x[index].squeeze().view(sizws[0], -1) * (1-lam)).view(sizws[0], sizws[1], sizws[2], sizws[3],)
        out = (x * lam + x[index].squeeze() * (1-lam))
        return out.float(), {'lam': lam, 'index': index}

    def init_weight(self):
        init_layer(self.fc1)

    def forward(self, input):        
        x = self.spectrogram_extractor(input)
        x = self.logmel_extractor(x)
        
        x_mels = self.logmel_extractor.power_to_db(x)
        x_pcen = self.pcen_converter(x) 
        x_clear = self.logmel_extractor.power_to_db(x ** 1.5)
        
        
        x = torch.cat((x_mels,x_pcen,x_clear),1)
        #x = torch.cat((x,x,x),1)
        
        frames_num = x.size(2)
        
        if self.training:
            x, mix_info = self.mixup(x)
            #x = self.spec_augmenter(x)
        else:
            mix_info = None
                
        # (batch_size, channels, freq, frames)
        x = self.base_model.extract_features(x)

        # (batch_size, channels, frames)
        x = torch.mean(x, dim=3)

        # channel smoothing
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        framewise_logit = interpolate(segmentwise_logit, self.interpolate_ratio)
        framewise_logit = pad_framewise_output(framewise_logit, frames_num)

        output_dict = {
            "framewise_output": framewise_output,
            "segmentwise_output": segmentwise_output,
            "logit": logit,
            "framewise_logit": framewise_logit,
            "clipwise_output": clipwise_output
        }

        return output_dict, mix_info

In [9]:
# LRAP. Instance-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = (scores.sum(-1) / labels.sum(-1)).mean()
    return score.item()

# label-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LWLRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    # Number of GT labels per instance
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()

def mixup_loss(cor, x, y, mix_info):
    clip_loss = cor(x['clipwise_output'], y) * mix_info['lam'] + cor(x['clipwise_output'], y[mix_info['index']].squeeze()) * (1-mix_info['lam'])
    frame_loss = cor(x['framewise_output'].max(dim=1)[0], y) * mix_info['lam'] + cor(x['framewise_output'].max(dim=1)[0], y[mix_info['index']].squeeze()) * (1-mix_info['lam'])
    return 0.5*clip_loss+frame_loss

def mixup_socre(cor, x, y, mix_info):
    return cor(x, y) * mix_info['lam'] + cor(x, y[mix_info['index']].squeeze()) * (1-mix_info['lam'])

In [10]:
train_tp = pd.read_csv('../../data/train_tp.csv')
train_fp = pd.read_csv('../../data/train_fp.csv')
submission = pd.read_csv('../../data/sample_submission.csv')

pred_target = list(submission.columns)[1:]
SR = settings['SR']

skf = StratifiedKFold(n_splits=settings['n_splits'], shuffle=True, random_state=42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
tta = np.zeros((len(submission), 24))
cv_score = 0

for fold_id, (train_index, val_index) in enumerate(skf.split(train_tp, train_tp.species_id)):
    print(f'---------- fold {fold_id} ----------')
    
    model = EfficientNetSED(**settings['model_params']).to(device)
    optim = Adam(model.parameters(), **settings['optim_params'])
    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optim, **scheduler_params)
    #scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optim, T_max=epochs-10)
    scheduler = wu.SimpleCosineAnnealingWarmup(optim, **settings['scheduler_params'])
    
    #pos_weights = torch.ones(24)
    #pos_weights = pos_weights * 24
    #criterion = nn.BCELoss()#pos_weight=pos_weights)
    #criterion = FocalLoss()
    criterion = cr.ImprovedFocalLoss(settings['loss_params'])
    
    train_dataset = RfcxDataSet(train_tp.iloc[train_index], **settings['train_params'])
    val_dataset   = RfcxDataSet(train_tp.iloc[val_index], **settings['val_params'])

    train_dataloader = DataLoader(train_dataset, shuffle=True, **settings['dataloder_params'])
    val_dataloader = DataLoader(val_dataset, shuffle=False, **settings['dataloder_params'])
    
    es = 50
    bast_score = 0
    for epoch in range(1, settings['epochs']):
        if es <= 0:
            break
        
        start_time = time.time()
        
        # train
        model.train()
        train_loss = 0
        train_score = 0
        train_corr = 0
        
        for data in train_dataloader:
            image = data[0].float().to(device)
            label = data[1]
            
            optim.zero_grad()
            output, mix_info = model(image)
            
            output = {k:v.cpu() for k,v in output.items()}
            pred_labels = output["framewise_output"].max(1)[0]
            
            loss = mixup_socre(criterion, output, label, mix_info)
            #loss = mixup_socre(criterion, output['clipwise_output'], label, mix_info)
            #loss = criterion(output[''], label, mix_info)
            score = mixup_socre(LWLRAP, pred_labels, label, mix_info)
            
            #score = LWLRAP(pred_labels, label)
                        
            loss.backward()
            optim.step()
            
            vals, answers = torch.max(pred_labels, 1)
            vals, targets = torch.max(label, 1)
            vals, targets2 = torch.max(label[mix_info['index']], 1)
            
            corrects = 0
            for i in range(0, len(answers)):
                if answers[i] == targets[i]:
                    corrects = corrects + 1
                if answers[i] == targets2[i]:
                    corrects = corrects + 1
                
                corrects = 1 if corrects > 0 else 0
                    
            
            train_corr += corrects
            train_loss += loss.item()
            train_score += score
            
        train_loss  /= len(train_dataloader)
        train_score /= len(train_dataloader)
        
        # val
        model.eval()
        val_loss = 0
        val_score = 0
        val_corr = 0

        with torch.no_grad():
            for val_data in val_dataloader:
                image = val_data[0].float().to(device)
                label = val_data[1]

                output, mix_info = model(image)
                output = {k:v.cpu() for k,v in output.items()}
                
                pred_labels = output["framewise_output"].max(1)[0]
                vals, answers = torch.max(pred_labels, 1)
                vals, targets = torch.max(label, 1)
                
                corrects = 0
                for i in range(0, len(answers)):
                    if answers[i] == targets[i]:
                        corrects = corrects + 1

                val_corr += corrects
                val_loss += criterion(output, label)
                val_score += LWLRAP(pred_labels, label)
                
        val_loss  /= len(val_dataloader)
        val_score /= len(val_dataloader)
        
        duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]
        print(f'E {epoch:3}| T | L: {train_loss:.3} | S: {train_score:.3} | C: {train_corr}/{len(train_dataset)} | V | L: {val_loss:.3} | S: {val_score:.3} | C: {val_corr}/{len(val_dataset)} | T: {duration} | es: {es}')

        if bast_score < val_score:
            bast_score = val_score
            bast_path = save(fold_id, model, optim, criterion)
        else:
            es -= 1
        
        if es <= 0:
            break
            
        scheduler.step(val_score)
    
    print(f"bast score: {bast_score}")
    cv_score += bast_score / settings['n_splits']
    del model, train_dataset, val_dataset, train_dataloader, val_dataloader, optim
    gc.collect()

---------- fold 0 ----------


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /home/yuigahama/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth



Loaded pretrained weights for efficientnet-b3
E   1| T | L: 3.1 | S: 0.229 | C: 65/972 | V | L: 1.96 | S: 0.211 | C: 17/244 | T: 0:00:30 | es: 50
E   2| T | L: 1.75 | S: 0.385 | C: 94/972 | V | L: 1.31 | S: 0.507 | C: 78/244 | T: 0:00:28 | es: 50
E   3| T | L: 1.48 | S: 0.535 | C: 97/972 | V | L: 1.11 | S: 0.652 | C: 121/244 | T: 0:00:29 | es: 50
E   4| T | L: 1.29 | S: 0.633 | C: 97/972 | V | L: 1.1 | S: 0.707 | C: 135/244 | T: 0:00:29 | es: 50
E   5| T | L: 1.17 | S: 0.685 | C: 97/972 | V | L: 0.838 | S: 0.761 | C: 156/244 | T: 0:00:29 | es: 50
E   6| T | L: 1.04 | S: 0.747 | C: 97/972 | V | L: 0.772 | S: 0.82 | C: 174/244 | T: 0:00:29 | es: 50
E   7| T | L: 0.991 | S: 0.754 | C: 98/972 | V | L: 0.689 | S: 0.847 | C: 183/244 | T: 0:00:28 | es: 50
E   8| T | L: 0.981 | S: 0.776 | C: 98/972 | V | L: 0.686 | S: 0.833 | C: 180/244 | T: 0:00:29 | es: 50
E   9| T | L: 0.864 | S: 0.8 | C: 97/972 | V | L: 0.766 | S: 0.818 | C: 173/244 | T: 0:00:29 | es: 49
E  10| T | L: 0.876 | S: 0.799 | C

In [12]:
print(cv_score)

0.9007556948661805


In [13]:
def prediction_for_clip(audio_id: str,
                        clip: np.ndarray, 
                        model: EfficientNetSED,
                        threshold=0.5):
    PERIOD = settings['duration']
    audios = []
    y = clip.astype(np.float32)
    len_y = len(y)
    start = 0
    end = PERIOD * SR
    while True:
        y_batch = y[start:end].astype(np.float32)

        start = int(end - ((settings['duration']/2) * SR))
        end += int((settings['duration']/2) * SR)
        
        #mel = create_mel(y_batch)
        audios.append(y_batch)
        
        if len_y < end:
            break
            
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    array = np.asarray(audios)
    image = torch.from_numpy(array).to(device)
    
    model.eval()
    estimated_event_list = []
    global_time = 0.0
    

    with torch.no_grad():
        prediction, _ = model(image)
        #frame_pred = torch.sum(
        #    torch.sigmoid(torch.max(prediction["framewise_output"], 1)[0]), 0
        #).detach().cpu().numpy()
        frame_pred = torch.max(
            torch.sigmoid(torch.max(prediction["framewise_output"], 1)[0]), 0
        )[0].detach().cpu().numpy()
    
    framewise_outputs = torch.max(prediction["framewise_output"], 0)[0].detach(
            ).cpu().numpy()
    thresholded = framewise_outputs >= threshold
    
    for target_idx in range(thresholded.shape[1]):
        if thresholded[:, target_idx].mean() == 0:
            pass
        else:
            detected = np.argwhere(thresholded[:, target_idx]).reshape(-1)
            head_idx = 0
            tail_idx = 0
            while True:
                if (tail_idx + 1 == len(detected)) or (
                        detected[tail_idx + 1] - 
                        detected[tail_idx] != 1):
                    onset = 0.01 * detected[
                        head_idx] + global_time
                    offset = 0.01 * detected[
                        tail_idx] + global_time
                    onset_idx = detected[head_idx]
                    offset_idx = detected[tail_idx]
                    max_confidence = framewise_outputs[
                        onset_idx:offset_idx, target_idx].max()
                    mean_confidence = framewise_outputs[
                        onset_idx:offset_idx, target_idx].mean()
                    estimated_event = {
                        "audio_id": audio_id,
                        "ebird_code": target_idx,
                        "onset": onset,
                        "offset": offset,
                        "max_confidence": max_confidence,
                        "mean_confidence": mean_confidence
                    }
                    estimated_event_list.append(estimated_event)
                    head_idx = tail_idx + 1
                    tail_idx = tail_idx + 1
                    if head_idx >= len(detected):
                        break
                else:
                    tail_idx += 1
        global_time += PERIOD
        
    prediction_df = pd.DataFrame(estimated_event_list)
    return prediction_df, frame_pred

In [14]:
def prediction(test_df: pd.DataFrame,
               model: dict,
               threshold=0.5):
    #model = get_model(model_config, weights_path)
    unique_audio_id = test_df.recording_id.unique()

    warnings.filterwarnings("ignore")
    prediction_dfs = []
    frame_dict = dict()
    for audio_id in tqdm(unique_audio_id):
        clip, _ = librosa.load(
            f'/home/yuigahama/kaggle/rfcx/data/test/{audio_id}.flac',
            sr=SR,
            mono=True,
            res_type="kaiser_fast"
        )
        
        test_df_for_audio_id = test_df.query(
            f"recording_id == '{audio_id}'").reset_index(drop=True)
        prediction_df, frame_pred = prediction_for_clip(
            audio_id,
            clip=clip,
            model=model,
            threshold=threshold
        )
        frame_dict[audio_id] = frame_pred
        prediction_dfs.append(prediction_df)
    
    prediction_df = pd.concat(prediction_dfs, axis=0, sort=False).reset_index(drop=True)
    return prediction_df, frame_dict

In [15]:
preds = []
frams = [] 
for i in range(5):

    model = EfficientNetSED(**settings['model_params']).to(device)
    params = torch.load(f'/home/yuigahama/kaggle/rfcx/model/{settings["test_name"]}/{settings["test_name"]}_{i}.model')
    model.load_state_dict(params['model_state_dict'])
    
    prediction_df, frame_dict = prediction(
        test_df=submission,
        model=model,
        threshold=0.5
    )
    print(len(prediction_df.audio_id.unique()))
    preds.append(prediction_df)
    frams.append(pd.DataFrame(frame_dict).T)

  0%|          | 0/1992 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b3


100%|██████████| 1992/1992 [04:30<00:00,  7.36it/s]


1982


  0%|          | 0/1992 [00:00<?, ?it/s]

...

1969


  0%|          | 0/1992 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b3


100%|██████████| 1992/1992 [04:24<00:00,  7.52it/s]


1991


In [16]:
sub = pd.DataFrame(np.zeros((len(submission), 24)),columns=pred_target, index=submission['recording_id'])
for p,j in zip(frams, [0.2, 0.2, 0.2, 0.2, 0.2]):
    p.columns = pred_target
    sub += (p * j)
#sub /= 5
sub.reset_index().to_csv(f'/home/yuigahama/kaggle/rfcx/model/{settings["test_name"]}/submission_{settings["test_name"]}.csv', index=False)
sub

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
recording_id,,,,,,,,,,,,,,,,,,,,,
000316da7,0.531943,0.532476,0.525788,0.712948,0.539988,0.535337,0.547825,0.519764,0.523199,0.547926,...,0.566363,0.532297,0.551605,0.536733,0.560100,0.524496,0.527276,0.531909,0.524489,0.537030
003bc2cb2,0.518713,0.528052,0.552478,0.680401,0.528862,0.522717,0.532107,0.527370,0.523369,0.537265,...,0.546760,0.529784,0.719174,0.531167,0.530730,0.529423,0.522770,0.539409,0.529637,0.526530
0061c037e,0.553309,0.535611,0.543160,0.656539,0.531518,0.543038,0.528235,0.591482,0.527890,0.574151,...,0.534398,0.542858,0.534859,0.578407,0.529479,0.544044,0.571685,0.526785,0.545354,0.552720
010eb14d3,0.717078,0.519537,0.522351,0.532216,0.528734,0.524687,0.518158,0.521941,0.670366,0.526674,...,0.524852,0.521584,0.522283,0.521124,0.571172,0.521969,0.518510,0.525738,0.515810,0.526480
011318064,0.519360,0.536918,0.551027,0.610418,0.518847,0.539205,0.547644,0.527252,0.517493,0.534610,...,0.715204,0.697109,0.538662,0.549409,0.595357,0.531968,0.526410,0.527585,0.523869,0.526612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff68f3ac3,0.599366,0.529214,0.552534,0.618460,0.547947,0.686292,0.522977,0.550688,0.529205,0.524240,...,0.529365,0.705460,0.528687,0.530279,0.582982,0.530508,0.543850,0.530095,0.527479,0.703976
ff973e852,0.541711,0.527770,0.530107,0.638689,0.525008,0.563038,0.600791,0.700004,0.539925,0.662495,...,0.540454,0.689335,0.528648,0.645156,0.532524,0.556479,0.604044,0.526327,0.543275,0.579172
ffa5cf6d6,0.534093,0.619661,0.590647,0.691530,0.520706,0.529345,0.533545,0.535488,0.521290,0.663771,...,0.541352,0.713161,0.530858,0.662032,0.521255,0.566800,0.535359,0.521899,0.526814,0.543256


In [17]:
len(np.arange(7e-5,8.0e-4,8e-5).tolist())

10

In [18]:
len(np.arange(1e-5,1.0e-3,7e-5).tolist())

15

In [19]:
print(torch.tensor([[1],[2],[3]]).size())
a*torch.tensor([[1],[2],[3]])

torch.Size([3, 1])


NameError: name 'a' is not defined

In [2]:
import numpy as np

In [12]:
np.arange(0.00001,0.00007,0.000012).tolist()

[1e-05, 2.2e-05, 3.4e-05, 4.599999999999999e-05, 5.7999999999999994e-05]